### Gradient Flow on $TS^2$ ###

Let $(q,v)\in TS^2$ be an element of the tangent bundle of the 2-dimensional sphere, embedded in $\mathbb{R}^3$.

We consider now the dynamical system resulting from the gradient of a given energy function

$$\dot{y} = -\nabla(E(q,v))$$


In [24]:
import autograd.numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from scipy.optimize import fsolve
from autograd import grad

In [28]:
def skw(x):
    return np.array([[0,-x[2],x[1]],[x[2],0,-x[0]],[-x[1],x[0],0]])

In [26]:
def energy(y):
    return 0.5*np.transpose(y[:3])@D@y[:3] + 0.5*np.transpose(y[3:])@M@y[3:]

def riemannGrad(y):
    myGrad = grad(energy)
    gE = myGrad(y)
    u = (np.cross(y[:3], gE[:3]) - np.transpose(y[:3])*gE[3:]*np.cross(y[:3], y[3:])).reshape(3,)
    v = np.cross(y[:3], gE[3:]).reshape(3,)
    return np.array([u, v]).reshape(6,)

def LieEulerStep(y0, y, dt):
    return actionSE3(expSE3(dt*riemannGrad(y)), y0)

def riemannDist(y, z):
    return 2*np.arcsin(np.linalg.norm(y-z,2)/2)

def expSE3(x):
    u = x[:3]
    v = x[3:]
    theta = np.linalg.norm(u)

    V = expSO3(u, theta)
    # tangent map on SO(3) and return all
    
def expSO3(x, om):
    return np.eye(3)+(np.sin(om)/om)*skw(x)+((1-np.cos(om))/om**2)*(skw(x)@skw(x))
    
def actionSE3(a, b):
    return np.array([a[:3,:3]@b[:3],a[:3,:3]@b[3:]+np.cross(a[3:,-1], a[:3,:3]@b[:3])])

In [27]:
I1 = np.random.rand(1)
I3 = np.random.rand(1)
if I1 > I3:
    tmp = np.copy(I3)
    I3 = np.copy(I1)
    I1 = np.copy(tmp)
I2 = np.copy(I1)
diag = np.array([1/I1, 1/I2, 1/I3]).reshape(3,)
D = np.diag(diag)

M = np.eye(3)

h = np.arange(0.1,10.0,0.1)
N = np.size(h)

# dimensions: method, initial condition, coordinate, time step size
YY = np.zeros((3,2,6,N+1))
RD = np.zeros((3,N+1))

# initialization
y0 = np.random.rand(2,3)
normy0 = np.linalg.norm(y0, axis=1)
y0[0,:] = y0[0,:]/normy0[0]
y0[1,:] = y0[1,:]/normy0[1]
YY[0,:,:3,0]=np.copy(y0)
YY[1,:,:3,0]=np.copy(y0)
YY[2,:,:3,0]=np.copy(y0)

my_fun = lambda x, x0, dt : - x + LieEulerStep(x0, x, dt)

for k in range(N):
    for j in range(2):
        YY[0,j,:,k+1] = LieEulerStep(YY[0,j,:,0],YY[0,j,:,0],h[k])
        YY[1,j,:,k+1] = fsolve(my_fun, YY[1,j,:,k], args=(YY[1,j,:,0], h[k]))
        # YY[2,j,:,k+1] = exact solution
    RD[0,k+1] = riemannDist(YY[0,0,:,k+1],YY[0,1,:,k+1])
    RD[1,k+1] = riemannDist(YY[1,0,:,k+1],YY[1,1,:,k+1])
    # RD[2,k+1] = riemannDist(YY[2,0,:,k+1],YY[2,1,:,k+1])


fig1 = plt.figure()
plt.plot(h, RD[0,1:], label='one step explicit Lie Euler')
plt.plot(h, RD[1,1:], label='one step implicit Lie Euler')
# plt.plot(h, RD[2,1:], label='exact solution')
plt.plot(h, RD[0,0]*np.ones((N, )), label='initial distance')
plt.legend()
plt.grid()

plt.show()

C:\Users\denis\AppData\Roaming\Python\Python310\site-packages\autograd\tracer.py:48: RuntimeWarning: invalid value encountered in arcsin
  return f_raw(*args, **kwargs)
C:\Users\denis\AppData\Roaming\Python\Python310\site-packages\scipy\optimize\_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\denis\AppData\Roaming\Python\Python310\site-packages\scipy\optimize\_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


ValueError: cannot convert float NaN to integer

In [21]:
c = np.array([[ 1, 2, 3],
              [-1, 1, 4]])

print(np.linalg.norm(c[0,:]/np.linalg.norm(c, axis=1)[0]))
print(np.linalg.norm(c[1,:]/np.linalg.norm(c, axis=1)[1]))

1.0
1.0
